In [5]:
# Quantum Boltzmann Machine for Route Optimisation
# ------------------------------------------------
# Fixed notebook implementing QBM-style route optimisation using given CSV files.

import math
import json
import random
from collections import defaultdict, Counter
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Quantum computing availability flags and imports (non-fatal)
DWAVE_AVAILABLE = False
QISKIT_AVAILABLE = False

# Try D-Wave / dimod imports
try:
    from dwave.system import DWaveSampler, EmbeddingComposite
    import dimod
    import dwave.inspector
    DWAVE_AVAILABLE = True
    print("D-Wave packages available")
except Exception as e:
    print(f"D-Wave packages not available: {e}")

# Try Qiskit / Aer imports
try:
    # qiskit Aer may be provided as qiskit_aer package in newer installs
    try:
        from qiskit_aer import Aer
    except Exception:
        # fallback to qiskit's Aer if available
        from qiskit import Aer
    from qiskit.primitives import Sampler
    from qiskit.circuit import QuantumCircuit
    from qiskit.circuit.library import RealAmplitudes
    from qiskit.algorithms.optimizers import SPSA
    QISKIT_AVAILABLE = True
    print("Qiskit packages available")
except Exception as e:
    print(f"Qiskit packages not available: {e}")

QUANTUM_IMPORTS_AVAILABLE = DWAVE_AVAILABLE or QISKIT_AVAILABLE
print(f"Quantum imports available: {QUANTUM_IMPORTS_AVAILABLE}")

D-Wave packages available
Qiskit packages not available: cannot import name 'Aer' from 'qiskit' (e:\Quantum Boltzmann Machine for Route Optimisation\.venv\Lib\site-packages\qiskit\__init__.py)
Quantum imports available: True


In [6]:
# Load and prepare data
try:
    # Load CSV files
    dist_df = pd.read_csv(r"C:\Users\Zwe Htet\Downloads\Large-Scale Route Optimization\distance.csv")
    orders_small = pd.read_csv(r"C:\Users\Zwe Htet\Downloads\Large-Scale Route Optimization\order_small.csv")
    orders_large = pd.read_csv(r"C:\Users\Zwe Htet\Downloads\Large-Scale Route Optimization\order_large.csv")
    print("Successfully loaded CSV files")

    # Build distance lookup
    def build_distance_lookup(df):
        d = {}
        for _, row in df.iterrows():
            a, b = str(row["Source"]), str(row["Destination"])
            w = float(row[df.columns[-1]])
            d[(a, b)] = min(d.get((a, b), w), w)
        for (a, b), w in list(d.items()):
            if (b, a) not in d:
                d[(b, a)] = w
        return d

    dist_lookup = build_distance_lookup(dist_df)
    print("Built distance lookup")

    # Choose a batch from orders_small
    if "Order_ID" not in orders_small.columns:
        batch_id = orders_small.iloc[0,0]
    else:
        batch_id = orders_small["Order_ID"].iloc[0]

    batch = orders_small[orders_small.iloc[:,0] == batch_id] if "Order_ID" not in orders_small.columns else orders_small[orders_small["Order_ID"] == batch_id]

    # find source and destinations
    source_city = str(batch["Source"].mode().iloc[0]) if "Source" in batch.columns else str(batch.iloc[0,1])
    destinations = sorted(set(batch["Destination"].astype(str))) if "Destination" in batch.columns else sorted(set(batch.iloc[:,2].astype(str)))

    cities = [source_city] + [c for c in destinations if c != source_city]
    N = len(cities)
    print(f"Found {N} cities to process")

except FileNotFoundError as e:
    print(f"Error loading CSV files: {str(e)}")
    raise

Successfully loaded CSV files
Built distance lookup
Found 2 cities to process
Built distance lookup
Found 2 cities to process


In [8]:
# Helper functions and solver (safe fallback to pure-Python SA)

def qubo_energy(Q, x):
    e = 0.0
    for (i, j), w in Q.items():
        e += w * float(x[i]) * float(x[j])
    return e


def onehot_from_route(route):
    x = np.zeros(N * N, dtype=int)
    for p, i in enumerate(route):
        x[var_index(i, p)] = 1
    return x


def decode_onehot_to_route(x):
    x_mat = x.reshape(N, N)
    chosen = np.argmax(x_mat, axis=0).tolist()
    seen, fixed = set(), []
    for c in chosen:
        if c not in seen:
            fixed.append(int(c)); seen.add(c)
        else:
            candidate = next(i for i in range(N) if i not in seen)
            fixed.append(int(candidate)); seen.add(candidate)
    return fixed


def simulated_annealing_local(Q, steps=10000):
    # Simple SA working directly on QUBO bitstring
    x = onehot_from_route(nearest_neighbour())
    e = qubo_energy(Q, x); best_x, best_e = x.copy(), e
    T = 1000.0
    for t in range(steps):
        i = np.random.randint(0, len(x))
        x_new = x.copy(); x_new[i] ^= 1
        e_new = qubo_energy(Q, x_new)
        if e_new < e or np.random.rand() < math.exp((e - e_new) / max(T, 1e-12)):
            x, e = x_new, e_new
            if e < best_e:
                best_x, best_e = x.copy(), e
        T *= 0.99995
    return best_x, best_e


def quantum_annealing_solve(Q, N):
    """Try D-Wave hardware; if not available, fallback to local SA."""
    # Try D-Wave hardware first (if available)
    if DWAVE_AVAILABLE:
        try:
            sampler = EmbeddingComposite(DWaveSampler())
            Q_dwave = {(i, j): float(w) for (i, j), w in Q.items()}
            response = sampler.sample_qubo(Q_dwave, num_reads=100)
            best_sample = response.first.sample
            x = np.array([best_sample.get(i, 0) for i in range(N * N)])
            return x, response.first.energy
        except Exception as e:
            print(f"D-Wave hardware attempt failed: {e}")
            print("Falling back to local simulated annealing...")
    else:
        print("D-Wave hardware not available; using local simulated annealing fallback")

    # Local simulated annealing fallback
    x, energy = simulated_annealing_local(Q, steps=20000)
    return x, energy


# Run solvers: greedy and fallback quantum (local SA)
print("Running solvers...")

# Greedy solution
def nearest_neighbour(start_idx: int = 0):
    """Return a route (list of city indices) using a nearest-neighbour greedy heuristic."""
    unvisited = set(range(N))
    route = []
    current = int(start_idx)
    route.append(current)
    unvisited.remove(current)
    while unvisited:
        # pick nearest unvisited city (ties broken by index)
        next_city = min(unvisited, key=lambda j: D[current, j])
        route.append(int(next_city))
        unvisited.remove(next_city)
        current = next_city
    return route

# Greedy solution
greedy = nearest_neighbour(0)
greedy_len = tour_length(greedy)
print(f"Greedy solution length: {greedy_len}")

# Attempt quantum / fallback
try:
    print("\nAttempting quantum annealing (or fallback)...")
    qa_x, qa_e = quantum_annealing_solve(Q, N)
    qa_route = decode_onehot_to_route(qa_x)
    qa_len = tour_length(qa_route)
    print(f"Quantum/fallback solution length: {qa_len}")
    results = pd.DataFrame({
        'method': ['Greedy', 'Quantum/Fallback'],
        'tour_length_m': [greedy_len, qa_len]
    })
    print(results)
except Exception as e:
    print(f"Solver failed: {e}")
    print("Only greedy result available")

Running solvers...
Greedy solution length: 4892187.0

Attempting quantum annealing (or fallback)...
D-Wave hardware attempt failed: API token not defined
Falling back to local simulated annealing...
Quantum/fallback solution length: 4892187.0
             method  tour_length_m
0            Greedy      4892187.0
1  Quantum/Fallback      4892187.0
Quantum/fallback solution length: 4892187.0
             method  tour_length_m
0            Greedy      4892187.0
1  Quantum/Fallback      4892187.0


In [9]:
# Set up the QUBO problem

def distance(a, b):
    return dist_lookup.get((a, b), 1e9) if a != b else 0.0

# Build distance matrix
D = np.zeros((N, N))
for i, a in enumerate(cities):
    for j, b in enumerate(cities):
        D[i, j] = distance(a, b)


def tour_length(route):
    return sum(D[int(route[i]), int(route[(i + 1) % len(route)])] for i in range(len(route)))


def var_index(i, p):
    return i * N + p

# Build TSP QUBO
def build_tsp_qubo(D, A=1000.0, B=1.0):
    Q = defaultdict(float)
    for p in range(N):
        for i in range(N):
            idx_i = var_index(i, p)
            Q[(idx_i, idx_i)] += -A
            for k in range(i + 1, N):
                idx_k = var_index(k, p)
                key = (min(idx_i, idx_k), max(idx_i, idx_k))
                Q[key] += 2.0 * A

    for i in range(N):
        for p in range(N):
            idx_i_p = var_index(i, p)
            Q[(idx_i_p, idx_i_p)] += -A
            for q in range(p + 1, N):
                idx_i_q = var_index(i, q)
                key = (min(idx_i_p, idx_i_q), max(idx_i_p, idx_i_q))
                Q[key] += 2.0 * A

    for p in range(N):
        p_next = (p + 1) % N
        for i in range(N):
            for j in range(N):
                idx_i_p = var_index(i, p)
                idx_j_next = var_index(j, p_next)
                key = (min(idx_i_p, idx_j_next), max(idx_i_p, idx_j_next))
                Q[key] += B * float(D[i, j])

    return Q

# Create QUBO matrix
Q = build_tsp_qubo(D)
print("QUBO matrix created successfully")

QUBO matrix created successfully
